In [1]:
from urlparse import urlparse
import pandas as pd
import csv
import json
import tldextract

ImportError: No module named tldextract

In [ ]:
def get_domain(url):
    if (pd.isnull(url)):
        return ""
    else:
        return urlparse(url.replace("www.","")).hostname

In [ ]:
def process_url(url):
    if (pd.isnull(url)):
        return ""
    else:
        return url.replace("https://", "").replace("http://", "").replace("www.","").rstrip('/')

In [ ]:
# import site list
site_csv = pd.read_csv('../final_sites.csv', names=['site'], header=None)
sites = site_csv['site'].apply(process_url).tolist()
site_domains = site_csv['site'].apply(get_domain).tolist()

In [ ]:
# import resource table
resources = pd.read_csv('input/3_col_resources.csv', names=['url', 'top_level_url', 'loading_origin'], header=None)
print len(resources)

# limit to resources whose site we can reasonably identify
resources['tlu_hn'] = resources['top_level_url'].apply(process_url)
resources['lo_hn'] = resources['loading_origin'].apply(process_url)
ident_resources = resources[(resources['tlu_hn'].isin(sites) | resources['lo_hn'].isin(sites))
]

# then attribute each resource to that site
# use loading origin 
source = ident_resources['lo_hn'].where(ident_resources['lo_hn'].isin(sites), ident_resources['tlu_hn'])
source = pd.DataFrame({'source': source})
ident_resources = pd.concat([ident_resources, source], axis=1)
ident_resources = ident_resources.drop(columns=['top_level_url', 'loading_origin', 'tlu_hn', 'lo_hn'])
print len(ident_resources)

# limit to third-party resources, or resources not originating from the source site
third_party_resources = ident_resources[~ident_resources['url'].apply(get_domain).isin(site_domains)]
print len(third_party_resources)

In [ ]:
# write to csv for ad analysis
third_party_resources.to_csv('output/third_party_resources.csv', index=False)

In [ ]:
# how many resources per site

In [ ]:
print third_party_resources.groupby(by='source').count().reset_index().sort_values(by='url', ascending=False)

In [ ]:
# how many domains per site

In [ ]:
def get_domain_for_org(url):
    extracted = tldextract.extract(url)
    return "{}.{}".format(extracted.domain, extracted.suffix)

In [ ]:
domains = pd.DataFrame({'domain': third_party_resources['url'].apply(get_domain_for_org)})
resource_domains = pd.concat([third_party_resources, domains], axis=1).drop(columns=['url'])
print resource_domains.groupby(by='source').domain.nunique()

In [ ]:
# which organizations do these domains represent

In [ ]:
with open('webXray/domain_owners.json') as f:
    domain_owners = json.load(f)
    
domain_to_org = dict()
for do in domain_owners:
    org = do['owner_name']
    
    for domain in do['domains']:
        domain_to_org[domain] = org

In [ ]:
domain_orgs = pd.DataFrame({'org': resource_domains['domain'].map(domain_to_org)})
resource_orgs = pd.concat([resource_domains, domain_orgs], axis=1).drop(columns=['domain'])
print resource_orgs.groupby(by='source').org.nunique()